In [1]:
using PyCall
using Distances
using StatsBase
using LinearAlgebra
using JuMP
using Gurobi
using CSV
using DataFrames
using SparseArrays
using Printf
using JSON

In [2]:
py"""
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import sys
import cv2
import os

class Triangulation:
    def load_imgs(self, path1, path2):
        self.img1 = cv2.imread(path1, cv2.CV_8UC3)
        self.img2 = cv2.imread(path2, cv2.CV_8UC3)
        # base = "/".join(path1.split("\\")[:-1])
        # cv2.imwrite(os.path.join(base, "left_loaded.png"), self.img1)
        # cv2.imwrite(os.path.join(base, "right_loaded.png"), self.img2)

    def findRootSIFTFeatures(self, n_components = None):
        class RootSIFT:
            def __init__(self):
                self.extractor = cv2.xfeatures2d.SIFT_create(n_components) if n_components != None else cv2.xfeatures2d.SIFT_create()
            def compute(self, image, kps, eps=1e-7):
                (kps, descs) = self.extractor.compute(image, kps)
                if len(kps) == 0:
                    return ([], None)

                descs /= (descs.sum(axis=1, keepdims=True) + eps)
                descs = np.sqrt(descs)
                return (kps, descs)

        class InnerFeatures:
            def __init__(self, kps, des, pos):
                self.kps = kps
                self.des = des
                self.pos = pos

        def innerRootSIFT(img):
            sift = cv2.xfeatures2d.SIFT_create(n_components) if n_components != None else cv2.xfeatures2d.SIFT_create()
            (kps, descs) = sift.detectAndCompute(img, None)

            rs = RootSIFT()
            (kps, descs) = rs.compute(img, kps)
            pos = [np.array([x.pt[0], x.pt[1]]) for x in kps]

            return kps, descs, pos

        kps1, desc1, pos1 = innerRootSIFT(self.img1)
        kps2, desc2, pos2 = innerRootSIFT(self.img2)
        self.feature_1 = InnerFeatures(kps1, desc1, pos1)
        self.feature_2 = InnerFeatures(kps2, desc2, pos2)
    def drawMatches(self, path):
        self.outImage = cv2.drawMatches(self.img1, self.feature_1.kps, self.img2, self.feature_2.kps, self.matches,outImg=None)
        cv2.imwrite(path, self.outImage)
scene = Triangulation()
"""
n_components = JSON.parse(String(read("../config\\config.json")))["config"]["SIFTFeatures"]

img1_path = "../data/dense/0000-small-left.png"
img2_path = "../data/dense/0001-small-right.png"

py"scene.load_imgs"(img1_path, img2_path)
py"scene.findRootSIFTFeatures"(n_components=n_components)

pts1 = py"scene.feature_1.pos"
pts2 = py"scene.feature_2.pos";

P_points = hcat(pts1...)'
Q_points = hcat(pts2...)'

println("size P points", size(P_points))
println("size Q points", size(Q_points))

cost = pairwise(Euclidean(), P_points, Q_points; dims=1)
println(size(cost))
P = ones(size(P_points,1))
Q = ones(size(Q_points,1));

size P points(100, 2)
size Q points(100, 2)
(100, 100)


In [3]:
solCount = 10
# m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0,PoolGap = 0.001))
m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0))

@variable(m, X[axes(cost,1), axes(cost,2)] ≥ 0, Int)
@objective(m, Min, cost ⋅ X)
@constraint(m,sum(X) .== min(sum(P), sum(Q)))
@constraint(m, X * ones(Int, length(Q)) .<= P)
@constraint(m, X'ones(Int, length(P)) .<= Q);
optimize!(m)
solution_pool = zeros(solCount, length(P),length(Q))
cnt = 0
obj = objective_value(m)

for i in 0:(solCount-1)
    try
        setparam!(m.moi_backend.inner,"SolutionNumber", i)
        xn = Gurobi.get_dblattrarray(m.moi_backend.inner, "Xn", 1, length(X))
        xn_val = Gurobi.get_dblattr(m.moi_backend.inner, "PoolObjVal")
#         if(round(xn_val,digits=1) != round(obj, digits=1))
        if(round(xn_val) != round(obj))
            println(cnt , " solution(s) selected")
            break
        end
        default = zeros(length(P),length(Q))
        for i in 0:length(P)-1
            default[i+1,:] = xn[(i*length(Q))+1:(i+1)*length(Q)]
        end
        solution_pool[i+1,:,:] = default
        cnt+=1
    catch 
        break
    end
end
sol_pool = deepcopy(solution_pool[1:cnt,:,:]);

Optimize a model with 201 rows, 10000 columns and 30000 nonzeros
Variable types: 0 continuous, 10000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.03s
Presolved: 201 rows, 10000 columns, 30000 nonzeros
Variable types: 0 continuous, 10000 integer (10000 binary)

Root relaxation: objective 5.728437e+03, 811 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5728.4372014 5728.43720  0.00%     -    0s
Optimal solution found at node 0 - now completing solution pool...
     0     0          -    0      5728.43720 5728.43720  0.00%     -    0s
     0     0          -    0      5728.43720 5728.43720  0.00%     -    0s
     0     2          -    0      5728.43720 5728.43720  0.00%     -    0s

In [9]:
n_sol = 6
solOther = sparse(sol_pool[n_sol,:,:])
sizeOf = min(size(P,1), size(Q,1))
matched_pts1 = zeros(sizeOf,2)
matched_pts2 = zeros(sizeOf,2)
i = 1
py"""
arr = []
"""
for (x,y,v) in zip(findnz(solOther)...)
    x_pos = [P_points'[:,x][1], Q_points'[:,y][1]]
    y_pos = [P_points'[:,x][2], Q_points'[:,y][2]]
    
    # dmatch creating
    queryId = x-1
    trainId = y-1
    distance = cost[x,y]
#     if(distance <= 10)
    dmatch = py"cv2.DMatch($(queryId), $(trainId),$(distance))"
    py"arr.append"(dmatch)
    matched_pts1[i,:] = [floor(x_pos[1]) floor(y_pos[1])]
    matched_pts2[i,:] = [floor(x_pos[2]) floor(y_pos[2])]
    i+=1
#     end
end
py"""
scene.matches = arr
"""


# path = "../data\\pair\\lastLPMatched.png"
# py"scene.drawMatches"(path)


matched_final_1 = deepcopy(matched_pts1[1:i-1, :])
matched_final_2 = deepcopy(matched_pts2[1:i-1, :]);
df = DataFrame()
df.PX = matched_final_1[:,1]
df.PY = matched_final_1[:,2]
df.QX = matched_final_2[:,1]
df.QY = matched_final_2[:,2];
print(size(df))

CSV.write("../data/dense/experiment/matchedPoints_$(n_sol).csv",  df, writeheader=false)

(17, 4)

"../data/dense/experiment/matchedPoints_6.csv"